In [ ]:
//#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"
#!eval-notebook "../Initialization/InitSystemorphToMemory"

In [ ]:
await Import.FromFile("../Files/Parameters/YieldCurveScenario.csv").WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

In [ ]:
await DataSource.DeleteAsync<YieldCurve>(await DataSource.Query<YieldCurve>().ToArrayAsync());
await DataSource.CommitAsync();

In [ ]:
    Activity.Start();
    var workspace = Workspace.CreateNew();
    workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
    
    var oldYieldCurves = await workspace.Query<YieldCurve>().ToArrayAsync();
    var parsingLog = await Import.FromFile("../Files/Parameters/YieldCurveScenario.csv").WithType<YieldCurve>().WithTarget(workspace).ExecuteAsync();  // TODO: FromDataSet(dataSet)
    if(parsingLog.Errors.Any()) return Activity.Finish().Merge(parsingLog); 
    var newYieldCurves = await workspace.Query<YieldCurve>().ToArrayAsync();
    
    newYieldCurves = newYieldCurves.Except(oldYieldCurves, YieldCurveComparer.Instance()).ToArray();
    if (!newYieldCurves.Any())
    {
        ApplicationMessage.Log(Warning.VariablesAlreadyImported, "");   
        return Activity.Finish().Merge(parsingLog);
    }
    await workspace.UpdateAsync<YieldCurve>(newYieldCurves.Concat(oldYieldCurves));
    await DataSource.UpdateAsync<YieldCurve>(newYieldCurves);
    await DataSource.CommitAsync();

    //var calculationLog = await CalculationYieldCurvesAsync(newYieldCurves);

    return Activity.Finish().Merge(parsingLog);//.Merge(calculationLog);

In [ ]:
newYieldCurves

In [ ]:
Activity.Start();
    var partitions = await DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync();
    var reportingNodeByPartition = await DataSource.Query<PartitionByReportingNode>().ToDictionaryAsync(x => x.Id, x => x.ReportingNode);

    var inactiveDataNodesByPeriodAndScenario = DataSource.Query<DataNodeState>()
                                .Where(x => x.State == State.Inactive)
                                .GroupBy(x => new Tuple<int,int,string> (x.Year, x.Month, x.Scenario).ToValueTuple())
                                .Select(x => new {Year = x.Key.Item1, 
                                                  Month = x.Key.Item2, 
                                                  Scenario = x.Key.Item3,
                                                  DataNodes = x.Select(y => y.DataNode)});

    var dataNodesByCurrencyAndReportingNode = DataSource.Query<DataNode>()
                                                .GroupBy(x => new Tuple<string, Guid>(x.FunctionalCurrency, x.Partition).ToValueTuple())
                                                .Select(x => new { Currency = x.Key.Item1, 
                                                                   ReportingNode = reportingNodeByPartition[x.Key.Item2], 
                                                                   DataNodes = x.Select(y => y.SystemName)});

    var updatableDataNodesByArgs = newYieldCurves.Join(dataNodesByCurrencyAndReportingNode, 
                                        yc => yc.Currency, dn => dn.Currency, 
                                        (yc, dn) => new { Args = new ImportArgs( dn.ReportingNode,
                                                                                 yc.Year, 
                                                                                 yc.Month,
                                                                                 Periodicity.Quarterly,
                                                                                 yc.Scenario,
                                                                                 ImportFormats.Cashflow), 
                                                          Currency = yc.Currency,
                                                          DataNodes = dn.DataNodes});
    
    foreach (var groupOfArgs in updatableDataNodesByArgs.GroupBy(x => new {ReportingNode = x.Args.ReportingNode, Scenario = x.Args.Scenario, Currency = x.Currency})) {
        var item = groupOfArgs.OrderBy(x => x.Args.Year).ThenBy(x => x.Args.Month).First();
        var newArgs = partitions.Where(x => (x.Scenario == item.Args.Scenario)  && x.ReportingNode == item.Args.ReportingNode &&
                                            (x.Year >= item.Args.Year || (x.Year == item.Args.Year && x.Month >= item.Args.Month)))
                                .Select(x => new ImportArgs(x.ReportingNode, x.Year, x.Month, Periodicity.Quarterly, x.Scenario, ImportFormats.Cashflow)).ToArray();
        updatableDataNodesByArgs = updatableDataNodesByArgs.Concat(newArgs.Select(x => new { Args = x, Currency = item.Currency, DataNodes = item.DataNodes }).ToArray()).ToArray();
    }
    
    // filter for active data nodes by period and scenario
    /*foreach(var inactiveDataNodes in inactiveDataNodesByPeriodAndScenario) {
        updatableDataNodesByArgs = updatableDataNodesByArgs.Where(x => x.Args.Year == inactiveDataNodes.Year && x.Args.Month == inactiveDataNodes.Month && x.Args.Scenario == inactiveDataNodes.Scenario)
                                .Select(x => new {Args = x.Args, Currency = x.Currency, DataNodes = x.DataNodes.Except(inactiveDataNodes.DataNodes)}).ToArray(); // TODO: proper testing
    }*/
                            
    var dataNodesToCalculation = updatableDataNodesByArgs.Distinct().OrderBy(x => x.Args.Year).ThenBy(x => x.Args.Month).ThenBy(x => x.Args.Scenario)
                                                         .Select(x => (Args: x.Args, Currency: x.Currency, DataNodes: x.DataNodes)).ToList();
    var initializationPartitionsLog = await InitializePartitionsAsync(dataNodesToCalculation);
    //var calculationPartitionsLog = await CalculateIfrsVariablesAsync(dataNodesToCalculation);
    return Activity.Finish().Merge(initializationPartitionsLog);//.Merge(calculationPartitionsLog);

In [ ]:
dataNodesToCalculation[0].Args == dataNodesToCalculation[1].Args

In [ ]:
await DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync()

In [ ]:
Activity.Start();
    foreach (var dataNodesInPartition in dataNodesToCalculation)
    {
        //var dataNodesInPartition = dataNodesToCalculation[2];
        var workspace = Workspace.CreateNew();
        workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
   
        var argsBestEstimate = new ImportArgs(dataNodesInPartition.Args.ReportingNode, dataNodesInPartition.Args.Year, dataNodesInPartition.Args.Month, Periodicity.Quarterly, null, ImportFormats.Cashflow);
        var defaultPartition = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate));
        var targetPartition = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(dataNodesInPartition.Args));
    
        await DataSource.Partition.SetAsync<PartitionByReportingNode>(dataNodesInPartition.Args);
        await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(dataNodesInPartition.Args);       
        var availableRawVars = await DataSource.Query<RawVariable>().Where(rv => dataNodesInPartition.DataNodes.Contains(rv.DataNode)).ToArrayAsync();
    
        if(!availableRawVars.Any() && defaultPartition != targetPartition)
        {
            await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(defaultPartition);      
            (availableRawVars.Concat(await DataSource.Query<RawVariable>().Where(rv => dataNodesInPartition.DataNodes.Contains(rv.DataNode)).ToArrayAsync()))
                        .Select(rv => rv with {Partition = targetPartition});
            await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(targetPartition);      
        }

        if(!availableRawVars.Any())
        {   
            ApplicationMessage.Log(Warning.MissingVariablesInPartition, argsBestEstimate.ReportingNode, 
                                    argsBestEstimate.Year.ToString(), argsBestEstimate.Month.ToString(), argsBestEstimate.Scenario);
            continue;
            //return Activity.Finish();
        }

        await workspace.UpdateAsync<RawVariable>(availableRawVars);
        var storage = new ImportStorage(dataNodesInPartition.Args, DataSource, workspace);
        await storage.InitializeAsync();
        if(Activity.HasErrors()) return Activity.Finish();

        var universe = Scopes.ForStorage(storage).ToScope<IModel>();
        var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
    
        var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
        if(Activity.HasErrors()) return Activity.Finish();
        
        await workspace.UpdateAsync<IfrsVariable>(ivs);
        await CommitToDatabase<IfrsVariable>(storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));  
        await CommitToDatabase<RawVariable>(storage.TargetPartition, 
                                        snapshot : true,
                                        filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
    }
    return Activity.Finish();

In [ ]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(defaultPartition);      
(availableRawVars.Concat(await DataSource.Query<RawVariable>().Where(rv => dataNodesInPartition.DataNodes.Contains(rv.DataNode)).ToArrayAsync())).Select(rv => rv with {Partition = targetPartition});

In [ ]:
availableRawVars

In [ ]:
//Import.DefineFormat(ImportFormats.YieldCurve, async (options, dataSet) => {
    Activity.Start();
    var workspace = Workspace.CreateNew();
    workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
    var oldYieldCurves = await workspace.Query<YieldCurve>().ToArrayAsync();
    
    var parsingLog = await Import.FromFile("../Files/Parameters/YieldCurveScenario.csv").WithType<YieldCurve>().WithTarget(workspace).ExecuteAsync();  // TODO: FromDataSet(dataSet)
    if(parsingLog.Errors.Any()) return Activity.Finish().Merge(parsingLog); 
    var newYieldCurves = await workspace.Query<YieldCurve>().ToArrayAsync();
    
    newYieldCurves = newYieldCurves.Except(oldYieldCurves, YieldCurveComparer.Instance()).ToArray();
    if (!newYieldCurves.Any())
    {
        ApplicationMessage.Log(Warning.VariablesAlreadyImported, "");   
        return Activity.Finish().Merge(parsingLog);
    }
    await workspace.UpdateAsync<YieldCurve>(newYieldCurves.Concat(oldYieldCurves));
    await DataSource.UpdateAsync<YieldCurve>(newYieldCurves);
    await DataSource.CommitAsync();

    Activity.Finish();
    

In [ ]:
    var workspace = Workspace.CreateNew();
    workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
   
    var partitions = await DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync();
    var reportingNodeByPartition = await DataSource.Query<PartitionByReportingNode>().ToDictionaryAsync(x => x.Id, x => x.ReportingNode);
    var inactiveDataNodesByPeriodAndScenario = DataSource.Query<DataNodeState>()
                                .Where(x => x.State == State.Inactive)
                                .GroupBy(x => new Tuple<int,int,string> (x.Year, x.Month, x.Scenario).ToValueTuple())
                                .Select(x => new {Year = x.Key.Item1, 
                                                  Month = x.Key.Item2, 
                                                  Scenario = x.Key.Item3,
                                                  DataNodes = x.Select(y => y.DataNode)});

    var dataNodesByCurrencyAndReportingNode = DataSource.Query<DataNode>()
                                                .GroupBy(x => new Tuple<string, Guid>(x.FunctionalCurrency, x.Partition).ToValueTuple())
                                                .Select(x => new { Currency = x.Key.Item1, 
                                                                   ReportingNode = reportingNodeByPartition[x.Key.Item2], 
                                                                   DataNodes = x.Select(y => y.SystemName)});

    var updatableDataNodesByArgs = newYieldCurves.Join(dataNodesByCurrencyAndReportingNode, 
                                        yc => yc.Currency, dn => dn.Currency, 
                                        (yc, dn) => new { Args = new ImportArgs( dn.ReportingNode,
                                                                                 yc.Year, 
                                                                                 yc.Month,
                                                                                 Periodicity.Quarterly,
                                                                                 yc.Scenario,
                                                                                 ImportFormats.Cashflow), 
                                                          Currency = yc.Currency,
                                                          DataNodes = dn.DataNodes }).ToArray();
    
    foreach (var groupOfArgs in updatableDataNodesByArgs.GroupBy(x => new {ReportingNode = x.Args.ReportingNode, Scenario = x.Args.Scenario, Currency = x.Currency})) {
        var item = groupOfArgs.OrderBy(x => x.Args.Year).ThenBy(x => x.Args.Month).First();
        var newArgs = partitions.Where(x => (x.Scenario == item.Args.Scenario)  && x.ReportingNode == item.Args.ReportingNode &&
                                            (x.Year >= item.Args.Year || (x.Year == item.Args.Year && x.Month >= item.Args.Month)))
                                .Select(x => new ImportArgs(x.ReportingNode, x.Year, x.Month, Periodicity.Quarterly, x.Scenario, ImportFormats.Cashflow)).ToArray();
        updatableDataNodesByArgs = updatableDataNodesByArgs.Concat(newArgs.Select(x => new { Args = x, Currency = item.Currency, DataNodes = item.DataNodes }).ToArray()).ToArray();
    }
    updatableDataNodesByArgs = updatableDataNodesByArgs.Distinct().ToArray();
    
    // filter for active data nodes by period and scenario
    foreach(var inactiveDataNodes in inactiveDataNodesByPeriodAndScenario) {
        updatableDataNodesByArgs = updatableDataNodesByArgs.Where(x => x.Args.Year == inactiveDataNodes.Year && x.Args.Month == inactiveDataNodes.Month && x.Args.Scenario == inactiveDataNodes.Scenario)
                                .Select(x => new {Args = x.Args, Currency = x.Currency, DataNodes = x.DataNodes.Except(inactiveDataNodes.DataNodes)}).ToArray(); // TODO: test properly
    }
                            
    updatableDataNodesByArgs = updatableDataNodesByArgs.OrderBy(x => x.Args.Year).ThenBy(x => x.Args.Month).ThenBy(x => x.Args.Scenario).ToArray();
    
updatableDataNodesByArgs.Select(x => (Args: x.Args, Currency: x.Currency, DataNodes: x.DataNodes)).ToList();
    updatableDataNodesByArgs

In [ ]:
    Activity.Start();
    foreach (var dataNodesInPartition in updatableDataNodesByArgs){
        var argsBE = new ImportArgs(dataNodesInPartition.Args.ReportingNode, dataNodesInPartition.Args.Year, dataNodesInPartition.Args.Month, Periodicity.Quarterly, null, ImportFormats.Cashflow);
        var beId = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(argsBE));
        var newPartitionWithPeriod = new PartitionByReportingNodeAndPeriod { 
                                   Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(dataNodesInPartition.Args)),
                                   Year = dataNodesInPartition.Args.Year,
                                   Month = dataNodesInPartition.Args.Month,
                                   ReportingNode = dataNodesInPartition.Args.ReportingNode, 
                                   Scenario = dataNodesInPartition.Args.Scenario };
        await DataSource.UpdateAsync<PartitionByReportingNodeAndPeriod>( new[]{ newPartitionWithPeriod, newPartitionWithPeriod with {Scenario = (string)null, Id = beId}});

        var newPartition = new PartitionByReportingNode { 
                                   Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNode>(dataNodesInPartition.Args)),
                                   ReportingNode = dataNodesInPartition.Args.ReportingNode};

        await DataSource.UpdateAsync<PartitionByReportingNode>( new[] { newPartition,
                newPartition with {Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNode>(argsBE))} } );  
        await DataSource.CommitAsync();
    }
    Activity.Finish()
    //TODO: what should happen when we import a YC for a scenario but the relative BE does not exist because it is not imported yet? and if we import the the Cf BE later?
    


In [ ]:
Activity.Start();
foreach (var dataNodesInPartition in updatableDataNodesByArgs){
   // var dataNodesInPartition = updatableDataNodesByArgs[2];
    var workspace = Workspace.CreateNew();
    workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
   
    var argsBestEstimate = new ImportArgs(dataNodesInPartition.Args.ReportingNode, dataNodesInPartition.Args.Year, dataNodesInPartition.Args.Month, Periodicity.Quarterly, null, ImportFormats.Cashflow);
    var defaultPartition = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate));
    var targetPartition = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(dataNodesInPartition.Args));
    
    await DataSource.Partition.SetAsync<PartitionByReportingNode>(dataNodesInPartition.Args);
    await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(dataNodesInPartition.Args);       
    var availableRawVars = await DataSource.Query<RawVariable>().Where(rv => dataNodesInPartition.DataNodes.Contains(rv.DataNode)).ToArrayAsync();
    
    if(!availableRawVars.Any() && defaultPartition != targetPartition)
    {
        await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(defaultPartition);      
        (availableRawVars.Concat(await DataSource.Query<RawVariable>().Where(rv => dataNodesInPartition.DataNodes.Contains(rv.DataNode)).ToArrayAsync()))
                        .Select(rv => rv with {Partition = targetPartition});
        await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(dataNodesInPartition.Args);      
    }

    if(!availableRawVars.Any())
    {   
        ApplicationMessage.Log(Warning.MissingVariablesInPartition, argsBestEstimate.ReportingNode, argsBestEstimate.Year.ToString(), argsBestEstimate.Month.ToString());
        //return Activity.Finish();
        continue;
    }

    await workspace.UpdateAsync<RawVariable>(availableRawVars);
    var storage = new ImportStorage(dataNodesInPartition.Args, DataSource, workspace);
    await storage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish().Merge(parsingLog);

    var universe = Scopes.ForStorage(storage).ToScope<IModel>();
    var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
    if(identities is null || !identities.Any()) ApplicationMessage.Log(Warning.Generic, "identities_null");
    
    var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
    if(Activity.HasErrors()) return Activity.Finish().Merge(parsingLog);
    if(ivs is null) ApplicationMessage.Log(Warning.Generic, "ivs_null");
        
    await workspace.UpdateAsync<IfrsVariable>(ivs);
    await CommitToDatabase<IfrsVariable>(storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));  
    await CommitToDatabase<RawVariable>(storage.TargetPartition, 
                                        snapshot : true,
                                        filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
}
return Activity.Finish();

In [ ]:
//reproduction of the Error: Message = Yield Curve not found for currency USD, year 2020, and month 12.
//Setting the workspace, creating my own importstorage
/*Activity.Start();
var args = updatableDataNodesByArgs[4].Args;
(int Year, int Month) CurrentReportingPeriod = (args.Year, args.Month);
(int Year, int Month) PreviousReportingPeriod = (args.Year - 1, 12);
var ImportFormat = args.ImportFormat;
var PartitionByRn = (await DataSource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).Single().Id;
var TargetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == CurrentReportingPeriod.Year &&
                                                                                                   p.Month == CurrentReportingPeriod.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
var DefaultPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                    p.Year == CurrentReportingPeriod.Year &&
                                                                                                    p.Month == CurrentReportingPeriod.Month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;
//Set Partitions
await DataSource.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
await workspace.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);

var parsedRawVariables = await workspace.Query<RawVariable>().ToArrayAsync();
var parsedIfrsVariables = await workspace.Query<IfrsVariable>().ToArrayAsync();
                
var DataNodeDataBySystemName = await workspace.LoadDataNodesAsync(args);

// Import Scopes and Data Node relationship parameters
var InterDataNodeParametersByGoc = await workspace.LoadInterDataNodeParametersAsync(args);
var primaryScopeFromParsedVariables = (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => x.DataNode) : parsedIfrsVariables.Select(x => x.DataNode)).ToHashSet();
var primaryScopeFromLinkedReinsurance = primaryScopeFromParsedVariables
                                            .Where(goc => !DataNodeDataBySystemName[goc].IsReinsurance && DataNodeDataBySystemName[goc].LiabilityType == LiabilityTypes.LRC)
                                            .SelectMany(goc => InterDataNodeParametersByGoc.TryGetValue(goc, out var interDataNodeParamByPeriod)
                                                                  ? interDataNodeParamByPeriod[CurrentPeriod].Select(param => param.DataNode == goc ? param.LinkedDataNode : param.DataNode).Where(goc => !primaryScopeFromParsedVariables.Contains(goc))
                                                                  : Enumerable.Empty<string>())
                                            .ToHashSet();
        
var primaryScope = primaryScopeFromParsedVariables.Concat(primaryScopeFromLinkedReinsurance).ToHashSet();
var secondaryScope = InterDataNodeParametersByGoc
                            .Where(kvp => primaryScope.Contains(kvp.Key))
                            .SelectMany(kvp => { var linkedGocs = kvp.Value[CurrentPeriod].Select(param => param.DataNode == kvp.Key ? param.LinkedDataNode : param.DataNode);
                                                return linkedGocs.Where(goc => !primaryScope.Contains(goc));}).ToHashSet();
var allImportScopes = new HashSet<string>(primaryScope.Concat(secondaryScope));
var importantDataNodes = allImportScopes.Select(dn => DataNodeDataBySystemName[dn]).ToArray();
var DataNodesByImportScope = new Dictionary<ImportScope, HashSet<string>> { { ImportScope.Primary, primaryScope }, { ImportScope.Secondary, secondaryScope } };
        
// Parameters
var PartnerRating = await workspace.LoadCurrentAndPreviousParameterAsync<PartnerRating>(args, x => x.Partner);
var CreditDefaultRates = await workspace.LoadCurrentAndPreviousParameterAsync<CreditDefaultRate>(args, x => x.CreditRiskRating);
var SingleDataNodeParametersByGoc = await workspace.LoadSingleDataNodeParametersAsync(args);
var LockedInYieldCurve = await workspace.LoadLockedInYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn]));
var CurrentYieldCurve = await workspace.LoadCurrentYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn])); //TODO Rename this variable
        
var AocConfigurationByAocStep = await DataSource.LoadAocStepConfigurationAsDictionaryAsync(args.Year, args.Month);         
var aocStepByInputSource = ((InputSource[])Enum.GetValues(typeof(InputSource))).ToDictionary(x => x,
                                                                                                 x => AocConfigurationByAocStep
                                                                                                 .Where(kvp => kvp.Value.InputSource.Contains(x))
                                                                                                 .Select(kvp => kvp.Key)
                                                                                                 .ToHashSet());
        
//Previous Period
var openingRawVariables = Enumerable.Empty<RawVariable>();
var openingIfrsVariables = Enumerable.Empty<IfrsVariable>();

if(allImportScopes.Select(dn => DataNodeDataBySystemName[dn]).Any(dnd => dnd.Year < args.Year)) {
    var PreviousPeriodPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                           p.Year == PreviousReportingPeriod.Year &&
                                                                                                           p.Month == PreviousReportingPeriod.Month &&
                                                                                                           p.Scenario == null).ToArrayAsync()).Single().Id;
            
    await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(PreviousPeriodPartition);
        
    //Perform queries to previous Period
    var openingRawVariables = (await DataSource.Query<RawVariable>()
                                   .Where(rv => rv.Partition == PreviousPeriodPartition && rv.AocType == AocTypes.CL)
                                   .Where(v => primaryScope.Contains(v.DataNode))
                                   .ToArrayAsync())
                .Select(rv => rv with {AocType = AocTypes.BOP, Novelty = Novelties.I, Values = rv.Values.Skip(MonthInAYear).ToArray(), Partition = TargetPartition});
            
    var openingIfrsVariables = (await DataSource.Query<IfrsVariable>()
                                    .Where(iv => iv.Partition == PreviousPeriodPartition && iv.AocType == AocTypes.EOP)
                                    .Where(v => allImportScopes.Contains(v.DataNode))
                                    .ToArrayAsync())
                .Select(iv => iv with {AocType = AocTypes.BOP, Novelty = Novelties.I, Partition = TargetPartition});
            
            // TODO: print error if 
            //openingRawVariables.Select(x => x.DataNode).ToHashSet() != dataNodesWithPreviousPeriod
    }
args*/

In [ ]:
CurrentYieldCurve

In [ ]:
Activity.Start();
var lockedInYieldCurveByGoc = new Dictionary<string, YieldCurve>();
foreach (var dn in importantDataNodes.Where(x => x.ValuationApproach == ValuationApproaches.BBA))
{
    var argsNew = args with {Year = dn.Year, Month = dn.Month, Scenario = dn.Scenario};
    
    var loadedYc = (await DataSource.LoadCurrentParameterAsync<YieldCurve>(argsNew, x => x.Currency, x => x.Currency == dn.ContractualCurrency));
        
        if(!loadedYc.TryGetValue(dn.ContractualCurrency, out var lockedYc))
            ApplicationMessage.Log(Error.YieldCurveNotFound, dn.ContractualCurrency, argsNew.Year.ToString(), argsNew.Month.ToString());
            
        lockedInYieldCurveByGoc[dn.DataNode] = lockedYc;
}
    
return lockedInYieldCurveByGoc;

return Activity.Finish();


In [ ]:
importantDataNodes


In [ ]:
var yieldCurve = (await workspace.LoadParameterAsync<YieldCurve>(args.Year, args.Month,  x => x.Currency == importantDataNodes[10].ContractualCurrency))
                             .Where(x => x.Scenario == args.Scenario || x.Scenario == null)
                             .GroupBy(x => x.Currency)
                             .Select(x => x.OrderByDescending(y => y.Year)
                                           .ThenByDescending(y => y.Month)
                                           .ThenByDescending(y => y.Scenario)
                                           .FirstOrDefault())
                             .ToDictionary(x => x.Currency);
yieldCurve

In [ ]:
universe.GetScope<ComputeAllScopes>(id).GetScope<ParentAocStep>((id, "PR")).Values

In [ ]:
await workspace.Partition.SetAsync<PartitionByReportingNode>(null);
await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(null);
await workspace.Query<YieldCurve>().ToArrayAsync()

In [ ]:
//await DataSource.Partition.SetAsync<PartitionByReportingNode>(null);
//await DataSource.LoadCurrentAndPreviousParameterAsync<DataNodeState>(args, x => x.DataNode)
//await DataSource.Query<PartitionByReportingNode>().ToArrayAsync()

In [ ]:
//DataNodeDataBySystemName

In [ ]:
//availableDataNodes.ToDictionary(kvp => dataNodesInPartition.DataNodes.Contains(kvp.Key), kvp => kvp.Value)

In [ ]:
/*//Storage debug
    
        var args = dataNodesInPartition.Args;
        var CurrentReportingPeriod = (args.Year, args.Month);
        var PreviousReportingPeriod = (args.Year - 1, MonthInAYear); // YTD Logic
        var ImportFormat = args.ImportFormat;
        var estimateTypes = (await workspace.Query<EstimateType>().ToArrayAsync());
        var hierarchyCache = workspace.ToHierarchicalDimensionCache();
        //end ad hoc variables
        var EstimateTypeDimension     = estimateTypes.ToDictionary(x => x.SystemName);
        var AmountTypeDimension       = (await workspace.Query<AmountType>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        var NoveltyDimension          = (await workspace.Query<Novelty>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        
        var ExperienceAdjustEstimateTypeMapping = new Dictionary<string,string>{{EstimateTypes.A, EstimateTypes.APA}, {EstimateTypes.AA, EstimateTypes.AAPA}, {EstimateTypes.OA, EstimateTypes.OAPA}}; //TODO move this logic
        
        //Hierarchy Cache
        hierarchyCache.Initialize<AmountType>();
        
        //EstimateType to load and to update
        var EstimateTypesByImportFormat = new InputSource[] { InputSource.Opening, InputSource.Actual, InputSource.Cashflow,}
                                        .ToDictionary(x => x.ToString(), 
                                                      x => estimateTypes
                                                      .Where(et => et.InputSource.Contains(x))
                                                      .Select(et => et.SystemName)
                                                      .ToHashSet());
        
        //ProjectionConfiguration : Current Period + projection for every Quarter End for current Year and next Years as in projectionConfiguration.csv
        var ProjectionConfiguration = (await workspace.Query<ProjectionConfiguration>()
                                               .Where(x => x.Shift > 0 || x.TimeStep == args.Month || (x.TimeStep > args.Month && x.TimeStep % 3 == 0))
                                               .OrderBy(x => x.Shift).ThenBy(x => x.TimeStep).ToArrayAsync());
        
        //Get Partitions
        var PartitionByRn = (await DataSource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).Single().Id;
        var TargetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == CurrentReportingPeriod.Year &&
                                                                                                   p.Month == CurrentReportingPeriod.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
        var DefaultPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                    p.Year == CurrentReportingPeriod.Year &&
                                                                                                    p.Month == CurrentReportingPeriod.Month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;
        //Set Partitions
        
        await DataSource.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        await workspace.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        
        await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        
        //Get data from Workspace (result of parsing)
        var parsedRawVariables = await workspace.Query<RawVariable>().ToArrayAsync();
        var parsedIfrsVariables = await workspace.Query<IfrsVariable>().ToArrayAsync();
        
        //DataNodes
        var DataNodeDataBySystemName = await workspace.LoadDataNodesAsync(args);
        var temp = await DataSource.LoadCurrentAndPreviousParameterAsync<DataNodeState>(args, x => x.DataNode);

        //Accident Years
        var AccidentYearsByDataNode = (IDictionary<string, ICollection<int?>>)
            (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => new {x.DataNode, x.AccidentYear}) : parsedIfrsVariables.Select(x => new {x.DataNode, x.AccidentYear}))
            .ToDictionaryGrouped(x => x.DataNode, x => (ICollection<int?>)x.Select(y => y.AccidentYear).ToHashSet());
        
        // Import Scopes and Data Node relationship parameters
        var InterDataNodeParametersByGoc = await workspace.LoadInterDataNodeParametersAsync(args);
        
        var primaryScopeFromParsedVariables = (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => x.DataNode) : parsedIfrsVariables.Select(x => x.DataNode)).ToHashSet();
        ApplicationMessage.Log(Warning.Generic, nameof(primaryScopeFromParsedVariables) + ": " + primaryScopeFromParsedVariables.Count().ToString());  
        var primaryScopeFromLinkedReinsurance = primaryScopeFromParsedVariables
                                            .Where(goc => !DataNodeDataBySystemName[goc].IsReinsurance && DataNodeDataBySystemName[goc].LiabilityType == LiabilityTypes.LRC)
                                            .SelectMany(goc => InterDataNodeParametersByGoc.TryGetValue(goc, out var interDataNodeParamByPeriod)
                                                                  ? interDataNodeParamByPeriod[CurrentPeriod].Select(param => param.DataNode == goc ? param.LinkedDataNode : param.DataNode).Where(goc => !primaryScopeFromParsedVariables.Contains(goc))
                                                                  : Enumerable.Empty<string>())
                                            .ToHashSet();
        
        var primaryScope = primaryScopeFromParsedVariables.Concat(primaryScopeFromLinkedReinsurance).ToHashSet();
        var secondaryScope = InterDataNodeParametersByGoc
                            .Where(kvp => primaryScope.Contains(kvp.Key))
                            .SelectMany(kvp => { var linkedGocs = kvp.Value[CurrentPeriod].Select(param => param.DataNode == kvp.Key ? param.LinkedDataNode : param.DataNode);
                                                return linkedGocs.Where(goc => !primaryScope.Contains(goc));}).ToHashSet();
        var allImportScopes = new HashSet<string>(primaryScope.Concat(secondaryScope));
        
        var DataNodesByImportScope = new Dictionary<ImportScope, HashSet<string>> { { ImportScope.Primary, primaryScope }, { ImportScope.Secondary, secondaryScope } };
        
        // Parameters
        var PartnerRating = await workspace.LoadCurrentAndPreviousParameterAsync<PartnerRating>(args, x => x.Partner);
        var CreditDefaultRates = await workspace.LoadCurrentAndPreviousParameterAsync<CreditDefaultRate>(args, x => x.CreditRiskRating);
        var SingleDataNodeParametersByGoc = await workspace.LoadSingleDataNodeParametersAsync(args);
        var LockedInYieldCurve = await workspace.LoadLockedInYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn]));
        var CurrentYieldCurve = await workspace.LoadCurrentYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn])); //TODO Rename this variable
        
        var AocConfigurationByAocStep = await DataSource.LoadAocStepConfigurationAsDictionaryAsync(args.Year, args.Month);         
        var aocStepByInputSource = ((InputSource[])Enum.GetValues(typeof(InputSource))).ToDictionary(x => x,
                                                                                                 x => AocConfigurationByAocStep
                                                                                                 .Where(kvp => kvp.Value.InputSource.Contains(x))
                                                                                                 .Select(kvp => kvp.Key)
                                                                                                 .ToHashSet());
        
    //ENd Storage Debug */

    /*await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(sceId);
    await workspace.UpdateAsync<RawVariable>(availableBERawVars.Select(x => 
            new RawVariable {Partition = sceId, 
                            Values = x.Values, 
                            DataNode = x.DataNode, 
                            AocType = x.AocType,
                            Novelty = x.Novelty,
                            AccidentYear = x.AccidentYear,
                            AmountType = x.AmountType,
                            EstimateType = x.EstimateType}));
    await workspace.UpdateAsync<RawVariable>(availableBERawVars.Select(x => 
            new RawVariable {Partition = beId, 
                            Values = x.Values, 
                            DataNode = x.DataNode, 
                            AocType = x.AocType,
                            Novelty = x.Novelty,
                            AccidentYear = x.AccidentYear,
                            AmountType = x.AmountType,
                            EstimateType = x.EstimateType}));
    await workspace.UpdateAsync<IfrsVariable>(availableBEIfrsVars.Concat(availableSceIfrsVars)); 
    await workspace.Partition.SetAsync<PartitionByReportingNode>(null);
    await DataSource.Partition.SetAsync<PartitionByReportingNode>(null);*/

    